In [7]:
import numpy as np
from matplotlib import pyplot as plt

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import Matern

In [ ]:
x = np.array([-2, 0, 4, 2]).reshape(-1,1)
y = np.array([1, 0.3, -1, 0]).reshape(-1,1)

x_test = np.linspace(-5, 5, 100).reshape(-1,1)

kernel = RBF(1, (1e-2, 1e2))
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9)

samples = gp.sample_y(x_test, 3)

#gp.fit(x, y)
#opt_params = gp.get_params()
#print "Opt param", opt_params

#log_lik = gp.log_marginal_likelihood()

# Make the prediction on the meshed x-axis (ask for MSE as well)
#y_pred, sigma = gp.predict(x_test, return_std=True)


In [ ]:
fig = plt.figure();
plt.plot(x_test, samples[:,0]);
plt.plot(x_test, samples[:,1]);
plt.plot(x_test, samples[:,2]);
#plt.fill_between(x_test, y_pred - 2*sigma, y_pred + 2*sigma, color="#dddddd")

Checking if we can estimate the length scale parameters

In [ ]:
x2_train = x_test
y2_train = samples[:,0]

kernel2 = RBF(10, (1e-2, 1e2))
print "Parameter before learning", kernel2.theta
gp2 = GaussianProcessRegressor(kernel=kernel2, n_restarts_optimizer=9, alpha=1e-06)
gp2 = gp2.fit(x2_train, y2_train)
opt_params = gp2.get_params()
print "Parameters after learning", gp2.kernel_.theta
print "Estimated l ", np.exp(gp2.kernel_.theta), "ground truth l ", 1

Example with multidimensional input

In [27]:
#Dimensions meaning
# 0 - (0/1)
# 1 - (0/1)
# 2 - int (positive: 1,2,3,4)
# 3 - {0,1,2}
# 4 - kappa (0.0 - 1.0)
Xtrain = np.array([
    [0,0,2,0,0.04],  #0
    [1,0,2,0,0.04],  #1
    [1,1,2,0,0.04],  #2
    [1,0,1,0,0.04],  #3
    [0,0,1,0,0.04],  #4
    [0,0,3,0,0.04],  #5
    [1,0,2,2,0.04],  #6
    [1,0,2,0,0.00],  #7
    [1,0,2,0,0.02],  #8
    [1,0,2,0,0.07]   #9
])
Ytrain = np.array([0.357,0.362,0.373,0.346,0.346, #0-4
                   0.340,0.3626,0.3034,0.3562,0.3654]) #5-9


l = np.full(5,10.0)
# se_kernel = RBF(l, (1e-5,1e2))
#se_kernel = Matern(l)
#se_kernel = Matern(l, nu=1.2)
print "Parameter before learning", np.exp(se_kernel.theta)


## learning part
gpReg = GaussianProcessRegressor(kernel=se_kernel, n_restarts_optimizer=9, alpha=1e-06)
gpReg = gpReg.fit(Xtrain, Ytrain)
print "Parameters after learning", np.exp(gpReg.kernel_.theta)

Parameter before learning [10. 10. 10. 10. 10.]
Parameters after learning [2.41172954e-03 2.10493426e+02 6.54424220e+01 7.73793273e+03
 8.54769136e-01]


In [28]:
Xtest = np.array([[1,1,2,0,0.01], #10
                  [1,1,2,2, 0.04] #11
                 ])

# expected output [0.358, 0.356]
y_pred, sigma = gpReg.predict(Xtest, return_std=True)
print y_pred

[0.34068581 0.37297982]
